### **Worldwide PM Polution and Related Mortality**

Link: https://www.kaggle.com/datasets/catiateixeira/wordwide-pm-polution-and-related-mortality

In [1]:
import pandas as pd
from scipy.stats import shapiro, levene, mannwhitneyu, kruskal, spearmanr
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from tabulate import tabulate

# Load the dataset
file_path = 'AirQuality.csv'
data = pd.read_csv(file_path)

# Print the data
print(tabulate(data, headers='keys', tablefmt='psql'))

+-----+------------------------+--------+-----------+-----------+---------------+------------+---------------+
|     | Country                |   Year |     PM2.5 |      PM10 |   TotalDeaths |   PMDeaths |   CitiesCount |
|-----+------------------------+--------+-----------+-----------+---------------+------------+---------------|
|   0 | Albania                |   2015 |  21.79    |  32.415   |      42.2998  |   21.7861  |             2 |
|   1 | Albania                |   2016 |  21.48    |  32.385   |      41.0188  |   20.8107  |             2 |
|   2 | Argentina              |   2015 |  10.26    |  27.87    |      33.0869  |   29.1591  |             1 |
|   3 | Australia              |   2010 |   8.04    |  15.3233  |      13.5717  |   13.1404  |             3 |
|   4 | Australia              |   2011 |   7.31667 |  14.4433  |      13.7276  |   13.2767  |             3 |
|   5 | Australia              |   2012 |   8.64    |  17.392   |      12.6597  |   12.1964  |             5 |
|

In [2]:
# Uji Asumsi: Normalitas, Homogenitas Varians, Multikolinearitas

# 1. Uji Normalitas
shapiro_total_deaths = shapiro(data['TotalDeaths'])
shapiro_pm_deaths = shapiro(data['PMDeaths'])

# 2. Uji Homogenitas Varians
levene_test = levene(data['TotalDeaths'], data['PMDeaths'])

# 3. Uji Multikolinearitas (VIF)
X = data[['PM2.5', 'PM10', 'CitiesCount']]
X = sm.add_constant(X)  # menambahkan konstanta

vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Menampilkan hasil uji asumsi
print("Shapiro-Wilk Test for TotalDeaths:", shapiro_total_deaths)
print("Shapiro-Wilk Test for PMDeaths:", shapiro_pm_deaths)
print("Levene Test for Homogeneity of Variances:", levene_test)
print("Variance Inflation Factors (VIF):\n", vif_data)

Shapiro-Wilk Test for TotalDeaths: ShapiroResult(statistic=np.float64(0.7969685505316983), pvalue=np.float64(4.289402602618286e-20))
Shapiro-Wilk Test for PMDeaths: ShapiroResult(statistic=np.float64(0.91073375604564), pvalue=np.float64(3.8312929077092457e-13))
Levene Test for Homogeneity of Variances: LeveneResult(statistic=np.float64(46.24177381002594), pvalue=np.float64(2.332980958781692e-11))
Variance Inflation Factors (VIF):
        feature       VIF
0        const  2.779282
1        PM2.5  3.983816
2         PM10  4.033629
3  CitiesCount  1.041110


In [3]:
# Transformasi Data

# Transformasi log untuk TotalDeaths dan PMDeaths
data['Log_TotalDeaths'] = np.log(data['TotalDeaths'] + 1)
data['Log_PMDeaths'] = np.log(data['PMDeaths'] + 1)

# Uji normalitas setelah transformasi
shapiro_log_total_deaths = shapiro(data['Log_TotalDeaths'])
shapiro_log_pm_deaths = shapiro(data['Log_PMDeaths'])

print("Shapiro-Wilk Test for Log_TotalDeaths:", shapiro_log_total_deaths)
print("Shapiro-Wilk Test for Log_PMDeaths:", shapiro_log_pm_deaths)

Shapiro-Wilk Test for Log_TotalDeaths: ShapiroResult(statistic=np.float64(0.9715475337385764), pvalue=np.float64(3.9162063763790304e-06))
Shapiro-Wilk Test for Log_PMDeaths: ShapiroResult(statistic=np.float64(0.9841379665711056), pvalue=np.float64(0.0009964483420827432))


In [4]:
# Uji Beda (Non-parametrik)

# Uji Mann-Whitney U untuk perbedaan berdasarkan negara (Contoh)
countries = data['Country'].unique()
country1 = data[data['Country'] == countries[0]]['Log_TotalDeaths']
country2 = data[data['Country'] == countries[1]]['Log_TotalDeaths']
mannwhitney_test = mannwhitneyu(country1, country2)

# Uji Kruskal-Wallis untuk perbedaan berdasarkan tahun
kruskal_test = kruskal(*[group['Log_TotalDeaths'].values for name, group in data.groupby('Year')])

print("Mann-Whitney U Test:", mannwhitney_test)
print("Kruskal-Wallis Test:", kruskal_test)

Mann-Whitney U Test: MannwhitneyuResult(statistic=np.float64(2.0), pvalue=np.float64(0.6666666666666666))
Kruskal-Wallis Test: KruskalResult(statistic=np.float64(4.565076665143806), pvalue=np.float64(0.7128674393895564))


In [5]:
# Uji korelasi Spearman
corr_pm25_totaldeaths = spearmanr(data['PM2.5'], data['Log_TotalDeaths'])
corr_pm10_totaldeaths = spearmanr(data['PM10'], data['Log_TotalDeaths'])

print("Spearman Correlation between PM2.5 and Log_TotalDeaths:", corr_pm25_totaldeaths)
print("Spearman Correlation between PM10 and Log_TotalDeaths:", corr_pm10_totaldeaths)

Spearman Correlation between PM2.5 and Log_TotalDeaths: SignificanceResult(statistic=np.float64(0.8339835041631326), pvalue=np.float64(1.7658144274378695e-87))
Spearman Correlation between PM10 and Log_TotalDeaths: SignificanceResult(statistic=np.float64(0.8337237660341695), pvalue=np.float64(2.2349255815316417e-87))


In [6]:
# Uji Regresi Berganda

# Model regresi berganda
X = data[['PM2.5', 'PM10', 'CitiesCount']]
y = data['Log_TotalDeaths']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# Ringkasan hasil regresi
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        Log_TotalDeaths   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.536
Method:                 Least Squares   F-statistic:                     128.9
Date:                Sun, 07 Jul 2024   Prob (F-statistic):           3.20e-55
Time:                        14:04:05   Log-Likelihood:                -212.82
No. Observations:                 333   AIC:                             433.6
Df Residuals:                     329   BIC:                             448.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           2.9795      0.042     70.705      